### 1.背景

对于Python来说，一切皆为对象，所有的变量赋值都遵循着对象引用机制。

程序在电脑上运行的时候，需要在内存中开辟出一块空间，用于存放运行时产生的临时变量，一方面，**内存的容量是有限的**；另一方面**内存空间的申请与回收是非常耗费精力的事情**，而且存在很大的危险性，稍有不慎就有可能引发内存溢出问题，好在Cpython解释器提供了自动的垃圾回收机制来帮我们解决了这件事。 

内存溢出：数据量过大时内存空间管理不善就很容易出现 **OOM(out of memory)**，俗称**爆内存**或者**内存溢出**，导致程序可能被操作系统中止。

内存泄漏：不是指你的内存在物理上消失了，也不是说你的内存被恶意程序利用出现了信息安全问题，而是指**程序本身没有设计好，未能及时释放已不再使用的内存，导致失去了对这段内存的控制，从而造成了内存的浪费（或者说这块内存脱离了垃圾回收机制的控制）**。

#### 1.1 什么是垃圾回收机制?

垃圾回收机制（简称GC）是 Python 解释器自带一种机，专门用来回收不可用的变量值所占用的内存空间

#### 1.2 为什么要有垃圾回收机制?

程序运行过程中会申请大量的内存空间，而对于一些无用的内存空间如果不及时清理的话会导致内存使用殆尽（内存溢出），导致程序崩溃，因此管理内存是一件重要且繁杂的事情，而python解释器自带的垃圾回收机制把程序员从繁杂的内存管理中解放出来

#### 1.3 什么时候进行垃圾回收？
由于变量名是访问到变量值的唯一方式，所以当一个变量值不再关联任何变量名时，我们就无法再访问到该变量值了，该变量值就是没有用的，就应该被当成一个垃圾回收。

#### 1.4 垃圾回收机制原理分析
 Python 的 GC 模块主要运用了**引用计数”（reference counting）** 来跟踪和回收垃圾。在引用计数的基础上，还可以通过“标记-清除”（mark and sweep）解决容器对象可能产生的循环引用的问题，并且通过“分代回收”（generation collection）以空间换取时间的方式来进一步提高垃圾回收的效率。 
 
 **用一句话来形容就是：引用计数机制为主，标记-清除和分代收集两种机制为辅的策略**

#### 1.5 垃圾回收机制的分类
- 引用计数
- 标记清楚
- 分代回收

### 2.引用计数机制

#### 2.1 引用计数法机制的原理

每个对象维护一个 ob_ref 字段，用来记录该对象当前被引用的次数，每当新的引用指向该对象时，它的引用计数 ob_ref 加 1，每当该对象的引用失效时计数 ob_ref 减 1，一旦对象的引用计数为 0，该对象立即被回收，对象占用的内存空间将被释放。


**缺点**
- 需要额外的空间维护引用计数
- 不能解决对象的循环引用

**优点**
- 高效，运行期没有停顿 可以类比一下Ruby的垃圾回收机制，也就是 
- 实时性：一旦没有引用，内存就直接释放。不用像其他机制等到特定时机。
- 处理回收内存的时间分摊到了平时
- 对象有确定的生命周期，而且易于实现

    
- **导致引用计数+1的情况**
    - 对象被创建，例如a=23
    - 对象被引用，例如b=a
    - 对象被作为参数，传入到一个函数中，例如func(a)
    - 对象作为一个元素，存储在容器中，例如list1=[a,a]

- **导致引用计数-1的情况**
    - 对象的别名被显式销毁，例如del a
    - 对象的别名被赋予新的对象，例如a=24
    - 一个对象离开它的作用域，例如:func函数执行完毕时，func函数中的局部变量（全局变量不会）
    - 对象所在的容器被销毁，或从容器中删除对象


#### 2.2 循环引用导致内存泄露

In [3]:
def f2():
    '''循环引用'''
    while True:
        c1=A()
        c2=A()
        print("c1引用次数:%s, c2引用次数:%s"%(sys.getrefcount(c1) - 1,sys.getrefcount(c2) - 1))
        c1.t=c2
        print("c1引用次数:%s, c2引用次数:%s"%(sys.getrefcount(c1) - 1,sys.getrefcount(c2) - 1))
        c2.t=c1
        print("c1引用次数:%s, c2引用次数:%s"%(sys.getrefcount(c1) - 1,sys.getrefcount(c2) - 1))
        del c1
        del c2
        break
        
f2()

object born id:0x1074a0b38
object born id:0x1074a0c50
c1引用次数:1, c2引用次数:1
c1引用次数:1, c2引用次数:2
c1引用次数:2, c2引用次数:2


- 创建了c1，c2后，这两个对象的引用计数都是1，执行c1.t=c2和c2.t=c1后，引用计数变成2.
- 在del c1后，内存c1的对象的引用计数变为1，由于不是为0，所以c1的对象不会被销毁,同理，在del c2后也是一样的。
- 虽然它们两个的对象都是可以被销毁的，但是由于循环引用，导致垃圾回收器都不会回收它们，所以就会导致内存泄露。
- 引用计数查看的代码:
```Python
import sys
sys.getrefcount(obj) - 1
```

### 3.标记清除

标记清除（Mark—Sweep）主要是回收循环引用的对象，显然，像是PyIntObject、PyStringObject这些不可变对象是不可能产生循环引用的，因为它们内部不可能持有其它对象的引用。Python中的循环引用总是发生在 **container** 对象之间，也就是能够在内部持有其它对象的对象，比如list、dict、class等等。这也使得该方法带来的开销只依赖于container对象的的数量。

原理解释：

首先标记对象（垃圾检测），然后清除垃圾（垃圾回收)，首先初始所有对象标记为白色，并确定根节点对象（这些对象是不会被删除），标记它们为黑色（表示对象有效）,将有效对象引用的对象标记为灰色（表示对象可达，但它们所引用的对象还没检查），检查完灰色对象引用的对象后，将灰色标记为黑色。重复直到不存在灰色节点为止。最后白色结点都是需要清除的对象。



标记清除底层机制：
- 收集所有的容器对象, 通过一个双向链表进行引用 （容器对象：列表、元祖、字典 ）
- 针对于每一个容器对象，通过一个变量gc_refs 来记录当前对应的引用计数
- 找到每个容器对象，找到它引用的容器对象，并将这个容器的引用计数-1
- 经过步骤3之后, 如果一个容器对象的引用计数为0， 就代表该对象可以被回收了, 循环引用导致其活到现在的

缺点：该机制所带来的额外操作和需要回收的内存块成正比。

### 4.分代回收
分代回收是一种**以空间换时间**的操作方式，Python将内存**根据对象的存活时间**划分为不同的集合，每个集合称为一个代，Python将内存分为了3“代”，分别为**年轻代（第0代）、中年代（第1代）、老年代（第2代）**，他们对应的是3个链表，它们的垃圾**收集频率随着对象存活时间的增大而减小**。

新创建的对象都会分配在年轻代，年轻代链表的总数达到上限时，Python垃圾收集机制就会被触发，把那些可以被回收的对象回收掉，而那些不会回收的对象就会被移到中年代去，依此类推，老年代中的对象是存活时间最久的对象，甚至是存活于整个系统的生命周期内。
同时，分代回收是建立在标记清除技术基础之上。

分代回收同样作为Python的辅助垃圾收集技术处理那些容器对象。

参考文档:
    - https://blog.51cto.com/u_15222272/5576251
    - https://blog.csdn.net/a6864657/article/details/128202002
    - http://testingpai.com/article/1630130800949